<a href="https://colab.research.google.com/github/elizabethlilies/KLBD2206014731/blob/main/cobaColab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install pyspark
!pip install pyspark
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark
# Import a Spark function from library
from pyspark.sql.functions import col

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 45.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a843c1699f31e7a1b0d12df2c5f5179db4d16fbbb0ceee704331626995ffd1cf
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
#menampilkan dataset yang dibutuhkan di Colab di bawah "Files" panel kiri
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

#**TUGAS**
Jika Anda berhasil menjalankan tahap penyiapan, Anda siap bekerja dengan kumpulan data 3 Juta Pesanan Instacart. Jika Anda ingin membaca lebih lanjut tentangnya, periksa posting blog resmi Instacart tentangnya, deskripsi skema ringkas dari kumpulan data, dan halaman unduhan.

Dalam Colab ini, kami hanya akan bekerja dengan kumpulan data pelatihan kecil (~131 ribu pesanan) untuk melakukan Penambangan Pola Sering yang cepat dengan algoritme FP-Growth.

In [30]:
#mengimport library yang dibutuhkan
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [4]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [5]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [6]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Gunakan API Spark Dataframe untuk menggabungkan 'produk' dan 'pesanan', sehingga Anda dapat melihat nama produk di setiap transaksi (dan bukan hanya id mereka). Kemudian, kelompokkan berdasarkan pesanan berdasarkan 'order_id' untuk mendapatkan satu baris per keranjang (yaitu, set produk yang dibeli bersama oleh satu pelanggan).

In [7]:
#memeriksa isi dari Products:
products.show(5)

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
+----------+--------------------+--------+-------------+
only showing top 5 rows



In [27]:
#memeriksa isi dari file 'Pesanan'
orders.show(5)

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
+--------+----------+-----------------+---------+
only showing top 5 rows



In [28]:
#menghitung jumlah orders yang ada
print("Terdapat {0} order".format(orders.count()))

Terdapat 1384617 order


In [31]:
#mengelompokkan order ID berdasarkan kolom order_id dan menghitung berapa banyak ID order yang ada:
IDorder = orders.groupBy("order_id").count().sort(desc("count"))
IDorder.show()

+--------+-----+
|order_id|count|
+--------+-----+
| 1395075|   80|
| 2813632|   80|
|  949182|   77|
|  341238|   76|
| 2869702|   76|
|  312611|   75|
| 1355077|   74|
| 1465173|   74|
|  653280|   72|
|  288915|   72|
| 1747908|   70|
| 2682845|   70|
|   45138|   70|
| 2395604|   70|
|  484152|   68|
| 1482636|   68|
| 2190647|   67|
|  428001|   66|
| 1388761|   66|
|  736120|   65|
+--------+-----+
only showing top 20 rows



In [19]:
#menggabungkan kedua dataframe sesuai id
#dengan dataframe API
ID_joined = products.join(orders, 
                              products.product_id == orders.product_id)
ID_joined.show()

+----------+--------------------+--------+-------------+--------+----------+-----------------+---------+
|product_id|        product_name|aisle_id|department_id|order_id|product_id|add_to_cart_order|reordered|
+----------+--------------------+--------+-------------+--------+----------+-----------------+---------+
|     49302|    Bulgarian Yogurt|     120|           16|       1|     49302|                1|        1|
|     11109|Organic 4% Milk F...|     108|           16|       1|     11109|                2|        1|
|     10246|Organic Celery He...|      83|            4|       1|     10246|                3|        0|
|     49683|      Cucumber Kirby|      83|            4|       1|     49683|                4|        0|
|     43633|Lightly Smoked Sa...|      95|           15|       1|     43633|                5|        1|
|     13176|Bag of Organic Ba...|      24|            4|       1|     13176|                6|        0|
|     47209|Organic Hass Avocado|      24|            4

In [21]:
#menampilkan barang yang dibeli oleh pelanggan dengan order_id 1
ID_1 = ID_joined.where("order_id = '1'")
ID_1.show()

+----------+--------------------+--------+-------------+--------+----------+-----------------+---------+
|product_id|        product_name|aisle_id|department_id|order_id|product_id|add_to_cart_order|reordered|
+----------+--------------------+--------+-------------+--------+----------+-----------------+---------+
|     49302|    Bulgarian Yogurt|     120|           16|       1|     49302|                1|        1|
|     11109|Organic 4% Milk F...|     108|           16|       1|     11109|                2|        1|
|     10246|Organic Celery He...|      83|            4|       1|     10246|                3|        0|
|     49683|      Cucumber Kirby|      83|            4|       1|     49683|                4|        0|
|     43633|Lightly Smoked Sa...|      95|           15|       1|     43633|                5|        1|
|     13176|Bag of Organic Ba...|      24|            4|       1|     13176|                6|        0|
|     47209|Organic Hass Avocado|      24|            4

In [34]:
#menampilkan barang yang dibeli oleh pelanggan dengan order_id 1395075 
ID_1395075 = ID_joined.where("order_id = '1395075'")
ID_1395075.show()

+----------+--------------------+--------+-------------+--------+----------+-----------------+---------+
|product_id|        product_name|aisle_id|department_id|order_id|product_id|add_to_cart_order|reordered|
+----------+--------------------+--------+-------------+--------+----------+-----------------+---------+
|      2495|Original Liquid L...|      75|           17| 1395075|      2495|                1|        1|
|     15460|To Go Stain Remov...|      75|           17| 1395075|     15460|                2|        0|
|      8366|Lemon Clean Furni...|     114|           17| 1395075|      8366|                3|        1|
|      5586|Original Glass Cl...|     114|           17| 1395075|      5586|                4|        1|
|      1142|Crystal Rain Trig...|     114|           17| 1395075|      1142|                5|        1|
|     26749|Power & Free Cool...|     114|           17| 1395075|     26749|                6|        1|
|     42406|Multi-Surface Cle...|     114|           17